In [1]:
import os
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend([os.path.abspath('..')])

Python 3.10.12 (main, Feb  4 2025, 14:57:36) [GCC 11.4.0] on linux


In [2]:
import torch.nn as nn
import torch
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader


In [63]:
hcp

,NEW_ID,Label,Short_label,Cortical Division,Cortical_Division_Number,X,Y,Z,ColeAnticevic_functional_network,ColeAnticevic_functional_network_label
HCPex_ID,,,,,,,,,,
1,1,Primary_Visual_Cortex_L,L_V1,Primary_Visual,1,-10.078,-84.159,1.759,1,Visual1
181,2,Primary_Visual_Cortex_R,R_V1,Primary_Visual,1,12.231,-80.644,4.101,1,Visual1
2,3,Second_Visual_Area_L,L_V2,Early_Visual,2,-12.044,-82.059,4.029,2,Visual2
182,4,Second_Visual_Area_R,R_V2,Early_Visual,2,12.367,-78.138,5.714,2,Visual2
3,5,Third_Visual_Area_L,L_V3,Early_Visual,2,-16.607,-84.793,7.259,2,Visual2
...,...,...,...,...,...,...,...,...,...,...
424,422,Mammillary_bodies_R,R_MB,Subcortical,24,2.396,-7.792,-14.783,14,Subcortical
392,423,Septal_nucleus_L,L_Septum,Subcortical,24,-2.371,6.267,-4.375,14,Subcortical
425,424,Septal_nucleus_R,R_Septum,Subcortical,24,1.183,6.383,-3.046,14,Subcortical


In [64]:
hcp = pd.read_excel('/home/tm/projects/OpenCloseBaseline/atlas/HCPex_Atlas_Description.xlsx',
                            index_col='HCPex_ID')
hcp.drop([337, 343, 344, 366], axis=0, inplace=True)

In [65]:
import os
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import pickle
from typing import Tuple, List, Optional, Callable
from nilearn.connectome import ConnectivityMeasure
import numpy as np




def load_rmet(strategy, atlas, GSR):

    sub = [f'{i+1:03d}' for i in range(64) if i!= 52]
    closed = fetch_ts('/data/Projects/OpenCloseChina/ts', sub=sub, GSR=GSR,
                      run=1, task='rest', strategy=strategy, atlas_name=atlas)
    opened = fetch_ts('/data/Projects/OpenCloseChina/ts', sub=sub, GSR=GSR,
                      run=2, task='rest', strategy=strategy, atlas_name=atlas)
    
    if atlas == 'HCPex':
        hcp = pd.read_excel('/home/tm/projects/OpenCloseBaseline/HCPex_Atlas_Description.xlsx',
                            index_col='HCPex_ID')
        
        to_del = [390]
        hcp.drop(to_del, axis=0, inplace=True)
        drop = [365, 372, 396, 398, 401, 405]

        for i in closed:
            i.columns = hcp.index
            i.drop(drop, axis=1, inplace=True)

        for i in opened:
            i.columns = hcp.index
            i.drop(drop, axis=1, inplace=True)
    
    closed = zscore(closed, axis=1, nan_policy='omit')
    opened = zscore(opened, axis=1, nan_policy='omit')

    #np.nan_to_num(closed, copy=False)
    #np.nan_to_num(opened, copy=False)

    n_closed, n_opened = closed.shape[0], opened.shape[0]

    #X = np.concatenate([closed, opened], axis=0)
    y = np.array([0] * n_closed + [1] * n_opened)
    groups = np.array([i for i in range(n_closed)] + 
                      [i for i in range(n_opened)])
    
    return closed, opened, y, groups

def load_ihb(strategy, atlas, GSR):

    coverage = np.load(f'/home/tm/projects/OpenCloseProject/coverage/ihb_{atlas}_parcel_coverage.npy') > 0.1

    sub = [f'{i+1:03d}' for i in range(84)]
    closed = fetch_ts('/data/Projects/OpenCloseIHB/outputs', sub=sub, GSR=GSR,
                      run=1, task='rest', strategy=strategy, atlas_name=atlas)
    opened = fetch_ts('/data/Projects/OpenCloseIHB/outputs', sub=sub, GSR=GSR,
                      run=2, task='rest', strategy=strategy, atlas_name=atlas)
    hcp = None
    if atlas == 'HCPex':
        hcp = pd.read_excel('/home/tm/projects/OpenCloseBaseline/atlas/HCPex_Atlas_Description.xlsx',
                            index_col='HCPex_ID')
        
        to_del = [365, 398, 401]
        hcp.drop(to_del, axis=0, inplace=True)
        drop = [372, 390, 396, 405]

        for i in closed:
            i.columns = hcp.index
            i.drop(drop, axis=1, inplace=True)

        for i in opened:
            i.columns = hcp.index
            i.drop(drop, axis=1, inplace=True)

        coverage = np.delete(coverage, to_del+drop)

    # shape = [sub, ts, roi]

    closed = np.array(closed)[:, :, coverage]
    opened = np.array(opened)[:, :, coverage]

    if atlas == 'HCPex':
        closed = np.delete(closed, [337, 343, 344, 366], axis=2)
        opened = np.delete(opened, [337, 343, 344, 366], axis=2)
        #hcp.drop([337, 343, 344, 366], axis=1, inplace=True)


    closed = zscore(closed, axis=1, nan_policy='omit') 
    opened = zscore(opened, axis=1, nan_policy='omit')

    n_closed, n_opened = closed.shape[0], opened.shape[0]

    #X = np.concatenate([closed, opened], axis=0)
    y = np.array([0] * n_closed + [1] * n_opened)
    groups = np.array([i for i in range(n_closed)] + 
                      [i for i in range(n_opened)])
    
    return closed, opened, y#, groups, hcp


def load_china(strategy, atlas, GSR):

    coverage = np.load(f'/home/tm/projects/OpenCloseProject/coverage/ihb_{atlas}_parcel_coverage.npy') > 0.1

    df = pd.read_csv(r"/arch/OpenCloseBeijin/BeijingEOEC.csv")

    open_ids2 = df['SubjectID'].loc[df['Session_2'] == 'open'].values
    open_ids3 = df['SubjectID'].loc[df['Session_3'] == 'open'].values

    closed_ids1 = df['SubjectID'].loc[df['Session_1'] == 'closed'].values
    closed_ids2 = df['SubjectID'].loc[df['Session_2'] == 'closed'].values
    closed_ids3 = df['SubjectID'].loc[df['Session_3'] == 'closed'].values

    closed1 = fetch_ts('/data/Projects/OpenCloseChina/outputs_china',
                  sub=closed_ids1, run=1, GSR=GSR,
                  atlas_name=atlas, strategy=strategy)

    closed2 = fetch_ts('/data/Projects/OpenCloseChina/outputs_china',
                        sub=closed_ids2, run=2, GSR=GSR,
                        atlas_name=atlas, strategy=strategy)

    closed3 = fetch_ts('/data/Projects/OpenCloseChina/outputs_china',
                        sub=closed_ids3, run=3, GSR=GSR,
                        atlas_name=atlas, strategy=strategy)

    opened2 = fetch_ts('/data/Projects/OpenCloseChina/outputs_china',
                        sub=open_ids2, run=2, GSR=GSR,
                        atlas_name=atlas, strategy=strategy)
    
    opened3 = fetch_ts('/data/Projects/OpenCloseChina/outputs_china',
                        sub=open_ids3, run=3, GSR=GSR,
                        atlas_name=atlas, strategy=strategy)
    
    

    for en, i in enumerate(closed3):
        if len(i) < 240:
            np.delete(closed_ids3, en)
            del closed3[en]
    for en, i in enumerate(opened2):
        if len(i) < 240:
            np.delete(open_ids2, en)
            del opened2[en]

    closed = closed1 + closed2 + closed3
    opened = opened2 + opened3

    #del opened[22]
    #del closed[84]

    if atlas == 'HCPex':
        hcp = pd.read_excel('/home/tm/projects/OpenCloseBaseline/atlas/HCPex_Atlas_Description.xlsx',
                            index_col='HCPex_ID')
        
        to_del = [365, 372, 396, 401, 405]
        hcp.drop(to_del, axis=0, inplace=True)
        drop = [390, 398]

        for i in closed:
            i.columns = hcp.index
            i.drop(drop, axis=1, inplace=True)
            
        for i in opened:
            i.columns = hcp.index
            i.drop(drop, axis=1, inplace=True)

        coverage = np.delete(coverage, to_del+drop)
            
    closed = np.array(closed)[:, :, coverage]
    opened = np.array(opened)[:, :, coverage]

    if atlas == 'HCPex':
        closed = np.delete(closed, [337, 343, 344, 366], axis=2)
        opened = np.delete(opened, [337, 343, 344, 366], axis=2)

    n_opened = opened.shape[0]

    closed = zscore(closed, axis=1, nan_policy='raise')[:n_opened]
    opened = zscore(opened, axis=1, nan_policy='raise')


    

    #op, cl = [], []
    #for en, i in enumerate(closed_ids1):
        #cl.append(en)
        #if i in open_ids2 or i in open_ids3:
            #op.append(en)
        #if i in closed_ids2 or i in closed_ids3:
            #cl.append(en)

    #del op[22]
    #del cl[84]

    #X = np.concatenate([closed, opened], axis=0)
    y = np.array([0] * n_opened + [1] * n_opened)
    #groups = np.array(cl + op)

    return closed, opened, y#, groups


def fetch_ts(path, sub=None, run=1, task='rest', strategy=4, atlas_name='AAL', GSR=False):
    ts = []
    if sub is None:
        sub = os.listdir(path)
    failed = []
    for i in sub:
        if not isinstance(i, str):
            i = str(i)
        if 'sub' in i:
            i = i[4:]
        try:
            name = (f'sub-{i}_task-{task}_run-{run}_time-series_{atlas_name}_strategy-{strategy}_GSR.csv' if GSR 
                    else f'sub-{i}_task-{task}_run-{run}_time-series_{atlas_name}_strategy-{strategy}.csv')
            path_to_file = os.path.join(path, f'sub-{i}', 'time-series', atlas_name, name)
            #print(path_to_file)
            ts.append(pd.read_csv(path_to_file))#.values)
        except FileNotFoundError:
            failed.append(i)
            continue

    if len(failed) > 0:
        print('no files available:', failed)
        
    return ts

In [60]:
#closed, opened, y = load_ihb(4, 'HCPex', True)
b = load_china(4, 'HCPex', True)

In [53]:
b[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [67]:
def get_split_ts(strategy, atlas, GSR):
    ihb_closed, ihb_opened, _ = load_ihb(strategy, atlas, GSR)
    china_closed, china_opened, china_y = load_china(strategy, atlas, GSR)

    train_idx, test_idx = train_test_split(
        np.arange(len(ihb_closed)), train_size=0.1, random_state=42)
    
    x_train = np.concatenate([ihb_closed[train_idx], ihb_opened[train_idx], 
                              china_closed[:, :120, :], china_opened[:, :120, :]], axis=0)
    
    y_train = np.concatenate([[0] * len(train_idx), [1] * len(train_idx), 
                              china_y], axis=0)
    
    x_test = np.concatenate([ihb_closed[test_idx], ihb_opened[test_idx]], axis=0)
    
    y_test = np.concatenate([[0] * len(test_idx), [1] * len(test_idx)], axis=0)
    
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = get_split_ts(4, 'HCPex', True)

In [68]:
class TimeDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.FloatTensor(data)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]
    

train_dataset = TimeDataset(x_train, y_train)
val_dataset = TimeDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
from model.logreg import LogisticRegression
model = LogisticRegression(x_train[1].shape[0], 1024, 2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 20
h = train(model, epochs, train_loader, val_loader, criterion, optimizer, scheduler=None, save_best=False, path_to_save=None)

In [70]:
from random import uniform, randint

import torch
from torch import nn



class ResidualBlock(nn.Module):
    """Residual block"""

    def __init__(self, block):
        super().__init__()
        self.block = block

    def forward(self, x: torch.Tensor):
        return self.block(x) + x


class MeanMLP(nn.Module):
    """
    meanMLP model for fMRI data.
    Expected input shape: [batch_size, time_length, input_feature_size].
    Output: [batch_size, n_classes]

    Hyperparameters expected in model_cfg:
        dropout: float
        hidden_size: int
        num_layers: int
    Data info expected in model_cfg:
        input_size: int - input_feature_size
        output_size: int - n_classes
    """

    def __init__(self, input_size, output_size, dropout, hidden_size, num_layers):
        super().__init__()


        # input block
        layers = [
            nn.Linear(input_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(p=dropout),
        ]
        # inter blocks: default HPs model has none of them
        for _ in range(num_layers):
            layers.append(
                nn.Sequential(
                    ResidualBlock(
                        nn.Sequential(
                            nn.Linear(hidden_size, hidden_size),
                            nn.LayerNorm(hidden_size),
                        )
                    ),
                    nn.ReLU(),
                    nn.Dropout(p=dropout),
                ),
            )

        # output block
        layers.append(
            nn.Linear(hidden_size, output_size),
        )

        self.fc = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor, introspection=False):
        bs, tl, fs = x.shape  # [batch_size, time_length, input_feature_size]

        fc_output = self.fc(x.view(-1, fs))
        fc_output = fc_output.view(bs, tl, -1)

        logits = fc_output.mean(1)

        if introspection:
            predictions = torch.argmax(logits, axis=-1)
            return fc_output, predictions

        return logits

In [71]:
model = MeanMLP(x_train[1].shape[1], 2, 0.1, 256, 2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 50
h = train(model, epochs, train_loader, val_loader, criterion, optimizer, scheduler=None, save_best=False, path_to_save=None)

  2%|▏         | 1/50 [00:00<00:13,  3.66it/s]

Epoch: 001, Train Loss: 1.5987, Test Loss 1.4942, Train Acc: 0.4799, Test Acc: 0.5250



  4%|▍         | 2/50 [00:00<00:11,  4.20it/s]

Epoch: 002, Train Loss: 0.6366, Test Loss 0.6859, Train Acc: 0.6975, Test Acc: 0.4875



  6%|▌         | 3/50 [00:00<00:10,  4.31it/s]

...

Epoch: 048, Train Loss: 0.0008, Test Loss 0.5133, Train Acc: 1.0000, Test Acc: 0.7625



 98%|█████████▊| 49/50 [00:12<00:00,  4.07it/s]

Epoch: 049, Train Loss: 0.0007, Test Loss 0.5127, Train Acc: 1.0000, Test Acc: 0.7625



100%|██████████| 50/50 [00:12<00:00,  3.98it/s]

Epoch: 050, Train Loss: 0.0006, Test Loss 0.5198, Train Acc: 1.0000, Test Acc: 0.7625

